In [1]:
import pandas as pd
import json
import numpy as np
import sklearn
import matplotlib.pyplot  as plt
import seaborn as sns
from datetime import datetime, date, time, timedelta
from dateutil.relativedelta import relativedelta

%matplotlib inline
sns.set(style='whitegrid')
pd.set_option('display.float_format', '{:.2f}'.format)

# Configure NumPy to avoid scientific notation
np.set_printoptions(suppress=True, precision=2)

In [7]:
def flatten_json(input_path, output_path):
    with open(input_path, 'r') as infile:
        # Read the entire JSON content
        content = infile.read()
        
        # Load the content as a nested list
        nested_list = json.loads(content)
        
        # Flatten the nested list
        flattened_list = [item for sublist in nested_list for item in sublist]
        
        # Write the flattened JSON to a new file
        with open(output_path, 'w') as outfile:
            json.dump(flattened_list, outfile, indent=4)

# Calling function to flatten the accounts JSON
input_file_path = '../data/test/accounts_data_test.json'
output_file_path = '../data/test/flattened_account_data.json'
flatten_json(input_file_path, output_file_path)

# Calling function to flatten the enquiry JSON
input_file_path = '../data/test/enquiry_data_test.json'
output_file_path = '../data/test/flattened_enquiry_data.json'
flatten_json(input_file_path, output_file_path)


In [11]:
#reading the files
account_df = pd.read_json('../data/test/flattened_account_data.json')
enquiry_df = pd.read_json('../data/test/flattened_enquiry_data.json')
flag_df = pd.read_csv('../data/test/test_flag.csv')

In [12]:
print("Size of account table - ",account_df.shape)
print("Size of enquiry table - ",enquiry_df.shape)
print("Size of flag table - ",flag_df.shape)

Size of account table -  (220013, 7)
Size of enquiry table -  (337662, 4)
Size of flag table -  (46127, 2)


## Inspecting flag_df 

In [13]:
flag_df.isna().sum()

uid                   0
NAME_CONTRACT_TYPE    0
dtype: int64

In [14]:
flag_df.duplicated().sum()

0

In [15]:
flag_df['NAME_CONTRACT_TYPE'].unique()

array(['Cash loans', 'Revolving loans'], dtype=object)

Preparing final dataset

In [18]:
final_df = flag_df.copy()

In [19]:
final_df = final_df.rename(columns={'NAME_CONTRACT_TYPE':'applied_contract_type'})
final_df.head()

,uid,applied_contract_type
0,CMO22835242,Cash loans
1,MRJ34316727,Cash loans
2,UAV00534378,Cash loans
3,IPQ08190402,Cash loans
4,NQN84331006,Cash loans


In [20]:
final_df.shape

(46127, 2)

## Inspecting account_df 

In [21]:
account_df.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid
0,Consumer credit,31630.50,0.00,2014-03-30,2014-11-29,000000000000000000000000,AAA14437029
1,Consumer credit,14613.39,0.00,2014-06-01,2014-11-03,000000000000000,AAA14437029
2,Credit card,54000.00,0.00,2015-12-13,2019-09-21,0000000000000000000000000000000000000000000000...,AAA14437029
3,Consumer credit,27076.50,0.00,2015-11-11,2016-11-24,000000000000000000000000000000000000,AAA14437029
4,Credit card,225000.00,0.00,2017-07-15,2019-11-14,0000000000000000000000000000000000000000000000...,AAA14437029


In [22]:
account_df.shape

(220013, 7)

In [23]:
account_df.duplicated().sum()

1104

In [24]:
#removing duplicates
account_df = account_df.drop_duplicates(keep='first')
account_df.shape

(218909, 7)

In [25]:
account_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 218909 entries, 0 to 220012
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   credit_type          218909 non-null  object 
 1   loan_amount          218909 non-null  float64
 2   amount_overdue       218909 non-null  float64
 3   open_date            218909 non-null  object 
 4   closed_date          137977 non-null  object 
 5   payment_hist_string  218909 non-null  object 
 6   uid                  218909 non-null  object 
dtypes: float64(2), object(5)
memory usage: 13.4+ MB


In [26]:
account_df['open_date'] = pd.to_datetime(account_df['open_date'])
account_df['closed_date'] = pd.to_datetime(account_df['closed_date'])

In [27]:
account_df.describe()

,loan_amount,amount_overdue,open_date,closed_date
count,218909.00,218909.00,218909,137977
mean,347219.89,29.40,2017-11-10 13:55:10.334430976,2018-03-10 18:15:56.976887296
min,0.00,0.00,2013-01-01 00:00:00,2013-01-08 00:00:00
25%,51340.50,0.00,2016-05-25 00:00:00,2016-11-12 00:00:00
50%,124650.00,0.00,2018-04-15 00:00:00,2018-07-12 00:00:00
75%,306000.00,0.00,2019-09-20 00:00:00,2019-10-28 00:00:00
max,58032000.00,959386.50,2020-12-31 00:00:00,2021-01-01 00:00:00
std,874910.42,3236.02,NaN,NaN


In [28]:
account_df[account_df['closed_date'].dt.date<account_df['open_date'].dt.date]

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid
56901,Mortgage,2250000.00,0.00,2017-07-02,2013-05-27,,GTR00709853


Removing these entries

In [29]:
remove_wrong_dates = account_df[account_df['closed_date'].dt.date<account_df['open_date'].dt.date].index
account_df = account_df.drop(remove_wrong_dates)
account_df.shape

(218908, 7)

Entries where loan_amount is 0

In [30]:
loan_amt_zero = account_df[account_df['loan_amount']==0]
loan_amt_zero

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid
44,Credit card,0.00,0.00,2015-09-21,2017-10-24,0000000000000000000000000000000000000000000000...,AAC85763409
105,Credit card,0.00,0.00,2019-08-23,NaT,000000000000000000000000000000000000000000000000,AAG67018395
140,Credit card,0.00,0.00,2018-11-03,NaT,0000000000000000000000000000000000000000000000...,AAK79175387
149,Credit card,0.00,0.00,2019-06-28,2020-06-27,000000000000000012042072000000000000,AAL42245978
158,Credit card,0.00,0.00,2019-06-28,NaT,0000000000000000000000000000000000000000000000...,AAL42245978
...,...,...,...,...,...,...,...
219875,Credit card,0.00,0.00,2019-09-16,NaT,000000000000000000000000000000000000000000000,ZZP02327859
219904,Credit card,0.00,0.00,2018-07-09,2018-08-26,000,ZZQ76221370
219942,Credit card,0.00,0.00,2020-07-12,2020-09-11,000000,ZZS89175663
219965,Credit card,0.00,0.00,2020-02-22,NaT,000000000000000000000000000000,ZZW35785310


In [31]:
account_df.drop(loan_amt_zero.index, inplace=True)

In [32]:
account_df.shape

(210841, 7)

#### Working on uid column

In [34]:
account_df['uid'].nunique()

39417

#### Working on credit_type column

In [35]:
account_df['credit_type'].unique()

array(['Consumer credit', 'Credit card', 'Microloan', 'Car loan',
       'Mortgage', 'Unknown type of loan',
       'Loan for business development', 'Another type of loan',
       'Loan for working capital replenishment',
       'Cash loan (non-earmarked)', 'Real estate loan',
       'Loan for the purchase of equipment'], dtype=object)

In [36]:
my_index = account_df[(account_df['credit_type']=='Another type of loan') | (account_df['credit_type']=='Unknown type of loan')].index

In [37]:
account_df.loc[my_index]

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid
266,Unknown type of loan,90000.00,0.00,2013-09-22,2015-08-01,0000000000000000000000000000000000000000000000...,AAS46317664
311,Unknown type of loan,675000.00,0.00,2014-01-18,NaT,0000000000000000000000000000000000000000000000...,AAX50423344
1845,Another type of loan,135000.00,0.00,2020-06-15,NaT,000000000008000000,AFW31156556
7060,Unknown type of loan,67500.00,0.00,2013-11-14,2014-05-24,000000000000000000,AWF93129012
10712,Unknown type of loan,585000.00,0.00,2020-05-02,NaT,000000000000000000000000,BHE26776901
...,...,...,...,...,...,...,...
208235,Unknown type of loan,101250.00,0.00,2014-06-29,2014-10-27,000000000000,YPP07327091
211045,Unknown type of loan,5043384.00,0.00,2016-10-30,2017-05-31,000000000000000000000,YYF12388564
211438,Unknown type of loan,83835.00,0.00,2013-08-02,2014-06-02,000000000000000000000000000000,YZN27891923
211868,Unknown type of loan,60435.00,0.00,2014-04-14,NaT,0000000000000000000000000000000000000000000000...,ZBF75136132


In [38]:
account_df['credit_type'] = account_df['credit_type'].replace(['Another type of loan','Unknown type of loan'],'other_loans')

In [41]:
loan_type_mapping = {
    'Consumer credit': 'consumer_loans',
    'Credit card': 'consumer_loans',
    'Mortgage': 'mortgages_and_real_estate_loans',
    'Car loan': 'automobile_loans',
    'Loan for business development': 'business_loans',
    'Microloan': 'consumer_loans',
    'Loan for working capital replenishment': 'business_loans',
    'Loan for the purchase of equipment': 'business_loans',
    'Real estate loan': 'mortgages_and_real_estate_loans',
    'Cash loan (non-earmarked)': 'consumer_loans',
    'Loan for purchase of shares (margin lending)': 'business_loans',
    'Interbank credit': 'business_loans',
    'Mobile operator loan': 'consumer_loans'
}

account_df['credit_type'] = account_df['credit_type'].replace(loan_type_mapping)

account_df.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid
0,consumer_loans,31630.50,0.00,2014-03-30,2014-11-29,000000000000000000000000,AAA14437029
1,consumer_loans,14613.39,0.00,2014-06-01,2014-11-03,000000000000000,AAA14437029
2,consumer_loans,54000.00,0.00,2015-12-13,2019-09-21,0000000000000000000000000000000000000000000000...,AAA14437029
3,consumer_loans,27076.50,0.00,2015-11-11,2016-11-24,000000000000000000000000000000000000,AAA14437029
4,consumer_loans,225000.00,0.00,2017-07-15,2019-11-14,0000000000000000000000000000000000000000000000...,AAA14437029


``` Making Total_tl feature ```

In [42]:
temp_df = account_df.groupby('uid').size().reset_index()
temp_df

,uid,0
0,AAA14437029,12
1,AAB12915377,3
2,AAB55088883,6
3,AAB68152393,7
4,AAC29580834,5
...,...,...
39412,ZZX89351447,6
39413,ZZX95870699,5
39414,ZZY22936172,8
39415,ZZY83980941,2


In [43]:
final_df = final_df.merge(temp_df, on ='uid', how='left')

In [44]:
final_df = final_df.rename(columns={0:'total_tl'})
final_df.head()

,uid,applied_contract_type,total_tl
0,CMO22835242,Cash loans,11.00
1,MRJ34316727,Cash loans,5.00
2,UAV00534378,Cash loans,5.00
3,IPQ08190402,Cash loans,5.00
4,NQN84331006,Cash loans,7.00


In [45]:
final_df.shape

(46127, 3)

``` Making closed TL and active TL columns```

In [46]:
temp_df = account_df[~account_df['closed_date'].isna()].groupby('uid').size().reset_index()
temp_df

,uid,0
0,AAA14437029,10
1,AAB12915377,1
2,AAB55088883,4
3,AAB68152393,5
4,AAC29580834,2
...,...,...
34337,ZZX56554441,5
34338,ZZX89351447,3
34339,ZZX95870699,3
34340,ZZY22936172,6


In [47]:
final_df = final_df.merge(temp_df, on ='uid', how='left')
final_df = final_df.rename(columns={0:'complete_tl'})
final_df['active_tl'] = final_df['total_tl']-final_df['complete_tl']
final_df.head()

,uid,applied_contract_type,total_tl,complete_tl,active_tl
0,CMO22835242,Cash loans,11.00,9.00,2.00
1,MRJ34316727,Cash loans,5.00,2.00,3.00
2,UAV00534378,Cash loans,5.00,3.00,2.00
3,IPQ08190402,Cash loans,5.00,3.00,2.00
4,NQN84331006,Cash loans,7.00,3.00,4.00


In [48]:
final_df.shape

(46127, 5)

``` Making total_amount_overdue, average_loan_amount, max_loan_amount and min_loan_amount columns ```

In [49]:
aggregated_df  = account_df.groupby('uid').agg({'amount_overdue':'sum','loan_amount': ['mean', 'min', 'max']}).reset_index()
aggregated_df.columns = ['uid', 'amount_overdue_sum', 'loan_amount_avg', 'loan_amount_min', 'loan_amount_max']
aggregated_df

,uid,amount_overdue_sum,loan_amount_avg,loan_amount_min,loan_amount_max
0,AAA14437029,0.00,311359.66,14613.39,1575000.00
1,AAB12915377,0.00,136002.66,45532.98,247500.00
2,AAB55088883,0.00,54565.75,4500.00,114471.90
3,AAB68152393,0.00,258105.21,63049.50,711000.00
4,AAC29580834,0.00,573507.36,40860.00,1422000.00
...,...,...,...,...,...
39412,ZZX89351447,0.00,306371.25,39186.00,1215000.00
39413,ZZX95870699,0.00,1288449.00,142245.00,2475000.00
39414,ZZY22936172,0.00,96957.03,32503.50,234000.00
39415,ZZY83980941,0.00,76844.25,37813.50,115875.00


In [50]:
final_df = pd.merge(final_df, aggregated_df, on='uid', how='left')

In [51]:
final_df.head()

,uid,applied_contract_type,total_tl,complete_tl,active_tl,amount_overdue_sum,loan_amount_avg,loan_amount_min,loan_amount_max
0,CMO22835242,Cash loans,11.00,9.00,2.00,0.00,939825.28,59233.50,6525000.00
1,MRJ34316727,Cash loans,5.00,2.00,3.00,0.00,353898.00,4500.00,972990.00
2,UAV00534378,Cash loans,5.00,3.00,2.00,0.00,425817.00,179550.00,703125.00
3,IPQ08190402,Cash loans,5.00,3.00,2.00,0.00,210009.60,60048.00,270000.00
4,NQN84331006,Cash loans,7.00,3.00,4.00,0.00,621868.68,135000.00,1978024.50


In [52]:
final_df.shape

(46127, 9)

``` Making num_times_delinquent column ```

In [53]:
account_df = account_df.reset_index(drop=True)

In [54]:
def calculate_payment_features(hist_string):
    segments = [int(hist_string[i:i+3]) for i in range(0, len(hist_string), 3)]
    # Calculate total deliquent payment (no. of months with any overdue days)
    num_times_delinquent = len([days for days in segments if days > 0])
    
    return num_times_delinquent

account_df['num_times_delinquent'] = account_df['payment_hist_string'].apply(calculate_payment_features)
account_df.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,num_times_delinquent
0,consumer_loans,31630.50,0.00,2014-03-30,2014-11-29,000000000000000000000000,AAA14437029,0
1,consumer_loans,14613.39,0.00,2014-06-01,2014-11-03,000000000000000,AAA14437029,0
2,consumer_loans,54000.00,0.00,2015-12-13,2019-09-21,0000000000000000000000000000000000000000000000...,AAA14437029,0
3,consumer_loans,27076.50,0.00,2015-11-11,2016-11-24,000000000000000000000000000000000000,AAA14437029,0
4,consumer_loans,225000.00,0.00,2017-07-15,2019-11-14,0000000000000000000000000000000000000000000000...,AAA14437029,10


In [55]:
deliquent_df = account_df.groupby('uid')['num_times_delinquent'].sum()

In [56]:
final_df.shape

(46127, 9)

In [57]:
final_df = pd.merge(final_df, deliquent_df, on='uid', how='left')

In [58]:
final_df.shape

(46127, 10)

In [59]:
final_df.sample(10)

,uid,applied_contract_type,total_tl,complete_tl,active_tl,amount_overdue_sum,loan_amount_avg,loan_amount_min,loan_amount_max,num_times_delinquent
21659,DGF56641640,Cash loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35850,VMQ43036179,Cash loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2928,YQK07763492,Cash loans,12.00,9.00,3.00,0.00,164329.88,20295.00,877342.50,0.00
45783,TKE75257522,Cash loans,2.00,1.00,1.00,0.00,316675.53,48351.06,585000.00,1.00
42875,NBU08306613,Revolving loans,3.00,1.00,2.00,0.00,166068.00,39249.00,382500.00,0.00
1129,BVL23617960,Cash loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11599,UDV67521700,Revolving loans,3.00,1.00,2.00,0.00,62278.50,45000.00,77377.50,0.00
42767,MDA72412836,Cash loans,1.00,NaN,NaN,0.00,247500.00,247500.00,247500.00,0.00
10838,PFY35349978,Cash loans,7.00,4.00,3.00,0.00,167831.36,25632.00,630000.00,0.00
24403,NVV76584566,Cash loans,2.00,2.00,0.00,0.00,142573.01,60146.01,225000.00,0.00


``` Making time since recent deliquency column ```

In [60]:
def find_recent_delinquency_date(hist_string, open_date):
    segments = [int(hist_string[i:i+3]) for i in range(0, len(hist_string), 3)]
    
    # Calculate time since recent delinquency
    recent_delinquency_index = None
    for i, days in enumerate(reversed(segments)):
        if days > 0:
            recent_delinquency_index = len(segments) - 1 - i
            break
    
    if recent_delinquency_index == None:
        time_since_recent_delinquency = None  # No delinquency found
    else:
        months_ago = len(segments) - 1 - recent_delinquency_index
        delinquency_date = open_date + relativedelta(months=months_ago)        

        return delinquency_date

def months_since_date(past_date):
    if pd.isna(past_date):
        return 0  # Return 0 if there is no delinquency
    today = datetime.today()
    diff = relativedelta(today, past_date)
    return diff.years * 12 + diff.months


account_df['recent_delinquency_date'] = account_df.apply(lambda row: find_recent_delinquency_date(row['payment_hist_string'], row['open_date']), axis=1)
account_df['months_since_recent_delinquency'] = account_df['recent_delinquency_date'].apply(months_since_date)

account_df.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,num_times_delinquent,recent_delinquency_date,months_since_recent_delinquency
0,consumer_loans,31630.50,0.00,2014-03-30,2014-11-29,000000000000000000000000,AAA14437029,0,NaT,0
1,consumer_loans,14613.39,0.00,2014-06-01,2014-11-03,000000000000000,AAA14437029,0,NaT,0
2,consumer_loans,54000.00,0.00,2015-12-13,2019-09-21,0000000000000000000000000000000000000000000000...,AAA14437029,0,NaT,0
3,consumer_loans,27076.50,0.00,2015-11-11,2016-11-24,000000000000000000000000000000000000,AAA14437029,0,NaT,0
4,consumer_loans,225000.00,0.00,2017-07-15,2019-11-14,0000000000000000000000000000000000000000000000...,AAA14437029,10,2017-08-15,84


In [61]:
account_df.head()

,credit_type,loan_amount,amount_overdue,open_date,closed_date,payment_hist_string,uid,num_times_delinquent,recent_delinquency_date,months_since_recent_delinquency
0,consumer_loans,31630.50,0.00,2014-03-30,2014-11-29,000000000000000000000000,AAA14437029,0,NaT,0
1,consumer_loans,14613.39,0.00,2014-06-01,2014-11-03,000000000000000,AAA14437029,0,NaT,0
2,consumer_loans,54000.00,0.00,2015-12-13,2019-09-21,0000000000000000000000000000000000000000000000...,AAA14437029,0,NaT,0
3,consumer_loans,27076.50,0.00,2015-11-11,2016-11-24,000000000000000000000000000000000000,AAA14437029,0,NaT,0
4,consumer_loans,225000.00,0.00,2017-07-15,2019-11-14,0000000000000000000000000000000000000000000000...,AAA14437029,10,2017-08-15,84


In [62]:
time_since_recent_deliquency_df = account_df.groupby('uid')['months_since_recent_delinquency'].min()

In [63]:
final_df.shape

(46127, 10)

In [64]:
final_df = pd.merge(final_df, time_since_recent_deliquency_df, on='uid', how='left')

In [65]:
final_df.shape

(46127, 11)

In [66]:
final_df.sample(10)

,uid,applied_contract_type,total_tl,complete_tl,active_tl,amount_overdue_sum,loan_amount_avg,loan_amount_min,loan_amount_max,num_times_delinquent,months_since_recent_delinquency
29741,ICY20873000,Cash loans,2.00,1.00,1.00,0.00,22778.68,22500.00,23057.37,3.00,0.00
42314,QXY93642497,Cash loans,10.00,6.00,4.00,0.00,379477.13,53244.00,985500.00,0.00,0.00
13433,QRA69165976,Cash loans,12.00,11.00,1.00,0.00,128481.79,27832.05,315000.00,3.00,0.00
22188,KOO47656412,Cash loans,4.00,3.00,1.00,0.00,51472.04,24943.14,112500.00,0.00,0.00
23511,UVX49028902,Cash loans,9.00,4.00,5.00,0.00,255362.17,58492.53,702000.00,0.00,0.00
4394,RPV81997295,Cash loans,11.00,9.00,2.00,0.00,156883.58,17806.50,662400.00,1.00,0.00
35857,VXL64899945,Cash loans,1.00,1.00,0.00,0.00,130950.00,130950.00,130950.00,0.00,0.00
9586,LDM58236856,Cash loans,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19566,PGB78809132,Cash loans,6.00,4.00,2.00,0.00,1030257.11,66492.68,2461050.00,0.00,0.00
33422,URQ27571986,Cash loans,3.00,2.00,1.00,0.00,186648.30,80558.82,337500.00,0.00,0.00


```Constructing frequency of each credit_type columns ```

In [67]:
for_pivot_df = account_df[['uid','credit_type']]
x = pd.pivot_table(for_pivot_df,index='uid',columns='credit_type',aggfunc='size', fill_value=0)
x.reset_index(inplace=True)
x

credit_type,uid,automobile_loans,business_loans,consumer_loans,mortgages_and_real_estate_loans,other_loans
0,AAA14437029,0,0,12,0,0
1,AAB12915377,0,0,3,0,0
2,AAB55088883,0,0,6,0,0
3,AAB68152393,0,0,7,0,0
4,AAC29580834,0,0,5,0,0
...,...,...,...,...,...,...
39412,ZZX89351447,0,0,6,0,0
39413,ZZX95870699,0,0,5,0,0
39414,ZZY22936172,0,0,8,0,0
39415,ZZY83980941,0,0,2,0,0


In [68]:
automobie_loan_count = x.groupby('uid')['automobile_loans'].sum()
business_loan_count = x.groupby('uid')['business_loans'].sum()
consumer_loan_count = x.groupby('uid')['consumer_loans'].sum()
real_estate_loan_count = x.groupby('uid')['mortgages_and_real_estate_loans'].sum()
other_loan_count = x.groupby('uid')['other_loans'].sum()

In [69]:
final_df = pd.merge(final_df, automobie_loan_count, on='uid', how='left')
final_df = pd.merge(final_df, business_loan_count, on='uid', how='left')
final_df = pd.merge(final_df, consumer_loan_count, on='uid', how='left')
final_df = pd.merge(final_df, real_estate_loan_count, on='uid', how='left')
final_df = pd.merge(final_df, other_loan_count, on='uid', how='left')

print(final_df.shape)

(46127, 16)


In [70]:
final_df.head()

,uid,applied_contract_type,total_tl,complete_tl,active_tl,amount_overdue_sum,loan_amount_avg,loan_amount_min,loan_amount_max,num_times_delinquent,months_since_recent_delinquency,automobile_loans,business_loans,consumer_loans,mortgages_and_real_estate_loans,other_loans
0,CMO22835242,Cash loans,11.00,9.00,2.00,0.00,939825.28,59233.50,6525000.00,1.00,0.00,1.00,0.00,9.00,1.00,0.00
1,MRJ34316727,Cash loans,5.00,2.00,3.00,0.00,353898.00,4500.00,972990.00,0.00,0.00,0.00,0.00,5.00,0.00,0.00
2,UAV00534378,Cash loans,5.00,3.00,2.00,0.00,425817.00,179550.00,703125.00,0.00,0.00,1.00,0.00,4.00,0.00,0.00
3,IPQ08190402,Cash loans,5.00,3.00,2.00,0.00,210009.60,60048.00,270000.00,0.00,0.00,0.00,0.00,5.00,0.00,0.00
4,NQN84331006,Cash loans,7.00,3.00,4.00,0.00,621868.68,135000.00,1978024.50,0.00,0.00,0.00,0.00,7.00,0.00,0.00


## Inspecting enquiry_df 

In [71]:
enquiry_df.head()

,enquiry_type,enquiry_amt,enquiry_date,uid
0,Car loan,143000,2020-12-13,AAA02107680
1,Real estate loan,174000,2020-12-01,AAA14437029
2,Loan for working capital replenishment,65000,2019-07-01,AAA14437029
3,Loan for working capital replenishment,118000,2020-08-05,AAA14437029
4,Car loan,12000,2020-02-28,AAA14437029


In [72]:
enquiry_df['enquiry_date'] = pd.to_datetime(enquiry_df['enquiry_date'])

In [74]:
enquiry_df.isna().sum()

enquiry_type    0
enquiry_amt     0
enquiry_date    0
uid             0
dtype: int64

In [75]:
enquiry_df.duplicated().sum()

0

In [76]:
enquiry_df = enquiry_df.drop_duplicates()

In [77]:
enquiry_df.shape

(337662, 4)

``` enquery_count and max_enquery_amount ```

In [78]:
aggregated_df  = enquiry_df.groupby('uid').agg({'enquiry_type':'count','enquiry_amt':'max'}).reset_index()
aggregated_df

,uid,enquiry_type,enquiry_amt
0,AAA02107680,1,143000
1,AAA14437029,4,174000
2,AAB12915377,1,137000
3,AAB55088883,14,176000
4,AAB68152393,17,186000
...,...,...,...
46122,ZZX89351447,2,105000
46123,ZZX95870699,2,177000
46124,ZZY22936172,15,184000
46125,ZZY83980941,5,158000


In [79]:
aggregated_df.rename(columns={'enquiry_type': 'enquiry_count', 'enquiry_amt': 'max_enquiry_amount'}, inplace=True)
final_df = pd.merge(final_df, aggregated_df, on='uid', how='left')
final_df.head()

,uid,applied_contract_type,total_tl,complete_tl,active_tl,amount_overdue_sum,loan_amount_avg,loan_amount_min,loan_amount_max,num_times_delinquent,months_since_recent_delinquency,automobile_loans,business_loans,consumer_loans,mortgages_and_real_estate_loans,other_loans,enquiry_count,max_enquiry_amount
0,CMO22835242,Cash loans,11.00,9.00,2.00,0.00,939825.28,59233.50,6525000.00,1.00,0.00,1.00,0.00,9.00,1.00,0.00,8,184000
1,MRJ34316727,Cash loans,5.00,2.00,3.00,0.00,353898.00,4500.00,972990.00,0.00,0.00,0.00,0.00,5.00,0.00,0.00,1,184000
2,UAV00534378,Cash loans,5.00,3.00,2.00,0.00,425817.00,179550.00,703125.00,0.00,0.00,1.00,0.00,4.00,0.00,0.00,1,188000
3,IPQ08190402,Cash loans,5.00,3.00,2.00,0.00,210009.60,60048.00,270000.00,0.00,0.00,0.00,0.00,5.00,0.00,0.00,21,192000
4,NQN84331006,Cash loans,7.00,3.00,4.00,0.00,621868.68,135000.00,1978024.50,0.00,0.00,0.00,0.00,7.00,0.00,0.00,6,170000


``` time_since_recent_enquiry ```

In [80]:
latest_enquiry_dates = enquiry_df.groupby('uid')['enquiry_date'].max().reset_index()
latest_enquiry_dates

,uid,enquiry_date
0,AAA02107680,2020-12-13
1,AAA14437029,2020-12-01
2,AAB12915377,2019-11-11
3,AAB55088883,2020-12-29
4,AAB68152393,2021-01-01
...,...,...
46122,ZZX89351447,2020-03-23
46123,ZZX95870699,2020-05-07
46124,ZZY22936172,2020-11-16
46125,ZZY83980941,2020-09-25


In [81]:
current_date = pd.Timestamp(datetime.now()) #present date
latest_enquiry_dates['time_since_last_enquiry'] = (current_date - latest_enquiry_dates['enquiry_date']).dt.days//30
latest_enquiry_dates.head()

,uid,enquiry_date,time_since_last_enquiry
0,AAA02107680,2020-12-13,45
1,AAA14437029,2020-12-01,45
2,AAB12915377,2019-11-11,58
3,AAB55088883,2020-12-29,44
4,AAB68152393,2021-01-01,44


In [82]:
latest_enquiry_dates_tomerge = latest_enquiry_dates[['uid','time_since_last_enquiry']]
final_df = pd.merge(final_df, latest_enquiry_dates_tomerge, on='uid', how='left')
final_df.head()

,uid,applied_contract_type,total_tl,complete_tl,active_tl,amount_overdue_sum,loan_amount_avg,loan_amount_min,loan_amount_max,num_times_delinquent,months_since_recent_delinquency,automobile_loans,business_loans,consumer_loans,mortgages_and_real_estate_loans,other_loans,enquiry_count,max_enquiry_amount,time_since_last_enquiry
0,CMO22835242,Cash loans,11.00,9.00,2.00,0.00,939825.28,59233.50,6525000.00,1.00,0.00,1.00,0.00,9.00,1.00,0.00,8,184000,50
1,MRJ34316727,Cash loans,5.00,2.00,3.00,0.00,353898.00,4500.00,972990.00,0.00,0.00,0.00,0.00,5.00,0.00,0.00,1,184000,76
2,UAV00534378,Cash loans,5.00,3.00,2.00,0.00,425817.00,179550.00,703125.00,0.00,0.00,1.00,0.00,4.00,0.00,0.00,1,188000,69
3,IPQ08190402,Cash loans,5.00,3.00,2.00,0.00,210009.60,60048.00,270000.00,0.00,0.00,0.00,0.00,5.00,0.00,0.00,21,192000,44
4,NQN84331006,Cash loans,7.00,3.00,4.00,0.00,621868.68,135000.00,1978024.50,0.00,0.00,0.00,0.00,7.00,0.00,0.00,6,170000,45


``` Frequency of each enquiry type ``` 

In [83]:
enquiry_df['enquiry_type'].unique()

array(['Car loan', 'Real estate loan',
       'Loan for working capital replenishment', 'Unknown type of loan',
       'Loan for the purchase of equipment', 'Mortgage',
       'Another type of loan', 'Cash loan (non-earmarked)',
       'Loan for business development', 'Cash loans', 'Microloan',
       'Credit card', 'Consumer credit', 'Revolving loans',
       'Mobile operator loan', 'Interbank credit',
       'Loan for purchase of shares (margin lending)'], dtype=object)

In [84]:
enquiry_type_mapping = {
    'Interbank credit' : 'business_loans_enquiries',
    'Mobile operator loan': 'consumer_loans_enquiries',
    'Cash loan (non-earmarked)': 'consumer_loans_enquiries',
    'Mortgage' : 'mortgages_and_real_estate_loans_enquiries',
    'Loan for purchase of shares (margin lending)': 'business_loans_enquiries',
    'Cash loans': 'consumer_loans_enquiries',
    'Car loan': 'automobile_loans_enquiries',
    'Another type of loan': 'other_loans_enquiries',
    'Loan for the purchase of equipment': 'business_loans_enquiries',
    'Real estate loan': 'mortgages_and_real_estate_loans_enquiries',
    'Credit card': 'consumer_loans_enquiries',
    'Unknown type of loan': 'other_loans_enquiries',
    'Microloan': 'consumer_loans_enquiries',
    'Consumer credit': 'consumer_loans_enquiries',
    'Loan for working capital replenishment': 'business_loans_enquiries',
    'Loan for business development': 'business_loans_enquiries',
    'Revolving loans': 'revolving_loans_enquiries'
}


enquiry_df['enquiry_type'] = enquiry_df['enquiry_type'].replace(enquiry_type_mapping)

In [85]:
enquiry_df['enquiry_type'].unique()

array(['automobile_loans_enquiries',
       'mortgages_and_real_estate_loans_enquiries',
       'business_loans_enquiries', 'other_loans_enquiries',
       'consumer_loans_enquiries', 'revolving_loans_enquiries'],
      dtype=object)

In [86]:
for_pivot_df = enquiry_df[['uid','enquiry_type']]
x = pd.pivot_table(for_pivot_df,index='uid',columns='enquiry_type',aggfunc='size', fill_value=0)
x.reset_index(inplace=True)
x

enquiry_type,uid,automobile_loans_enquiries,business_loans_enquiries,consumer_loans_enquiries,mortgages_and_real_estate_loans_enquiries,other_loans_enquiries,revolving_loans_enquiries
0,AAA02107680,1,0,0,0,0,0
1,AAA14437029,1,2,0,1,0,0
2,AAB12915377,0,0,0,1,0,0
3,AAB55088883,0,2,7,2,3,0
4,AAB68152393,0,7,5,1,2,2
...,...,...,...,...,...,...,...
46122,ZZX89351447,0,2,0,0,0,0
46123,ZZX95870699,0,0,2,0,0,0
46124,ZZY22936172,1,4,7,0,3,0
46125,ZZY83980941,0,1,3,1,0,0


In [87]:
business_loan_count = x.groupby('uid')['business_loans_enquiries'].sum()
consumer_loan_count = x.groupby('uid')['consumer_loans_enquiries'].sum()
revolving_loan_count = x.groupby('uid')['revolving_loans_enquiries'].sum()
real_estate_loan_count = x.groupby('uid')['mortgages_and_real_estate_loans_enquiries'].sum()
other_loan_count = x.groupby('uid')['other_loans_enquiries'].sum()
automobile_loan_count = x.groupby('uid')['automobile_loans_enquiries'].sum()

In [88]:
final_df = pd.merge(final_df, business_loan_count, on='uid', how='left')
final_df = pd.merge(final_df, consumer_loan_count, on='uid', how='left')
final_df = pd.merge(final_df, revolving_loan_count, on='uid', how='left')
final_df = pd.merge(final_df, real_estate_loan_count, on='uid', how='left')
final_df = pd.merge(final_df, other_loan_count, on='uid', how='left')
final_df = pd.merge(final_df, automobile_loan_count, on='uid', how='left')

print(final_df.shape)

(46127, 25)


In [89]:
#filling all NaN with 0
final_df.fillna(0, inplace=True)
final_df.head()

,uid,applied_contract_type,total_tl,complete_tl,active_tl,amount_overdue_sum,loan_amount_avg,loan_amount_min,loan_amount_max,num_times_delinquent,...,other_loans,enquiry_count,max_enquiry_amount,time_since_last_enquiry,business_loans_enquiries,consumer_loans_enquiries,revolving_loans_enquiries,mortgages_and_real_estate_loans_enquiries,other_loans_enquiries,automobile_loans_enquiries
0,CMO22835242,Cash loans,11.00,9.00,2.00,0.00,939825.28,59233.50,6525000.00,1.00,...,0.00,8,184000,50,3,4,0,0,1,0
1,MRJ34316727,Cash loans,5.00,2.00,3.00,0.00,353898.00,4500.00,972990.00,0.00,...,0.00,1,184000,76,0,0,0,0,0,1
2,UAV00534378,Cash loans,5.00,3.00,2.00,0.00,425817.00,179550.00,703125.00,0.00,...,0.00,1,188000,69,0,1,0,0,0,0
3,IPQ08190402,Cash loans,5.00,3.00,2.00,0.00,210009.60,60048.00,270000.00,0.00,...,0.00,21,192000,44,4,11,0,3,0,3
4,NQN84331006,Cash loans,7.00,3.00,4.00,0.00,621868.68,135000.00,1978024.50,0.00,...,0.00,6,170000,45,3,2,0,0,1,0


In [97]:
#to save memory
final_df['total_tl'] = final_df['total_tl'].astype('int64')
final_df['complete_tl'] = final_df['complete_tl'].astype('int64')
final_df['active_tl'] = final_df['active_tl'].astype('int64')
final_df['num_times_delinquent'] = final_df['num_times_delinquent'].astype('int64')
final_df['months_since_recent_delinquency'] = final_df['months_since_recent_delinquency'].astype('int64')
final_df['automobile_loans'] = final_df['automobile_loans'].astype('int64')
final_df['business_loans'] = final_df['business_loans'].astype('int64')
final_df['consumer_loans'] = final_df['consumer_loans'].astype('int64')
final_df['mortgages_and_real_estate_loans'] = final_df['mortgages_and_real_estate_loans'].astype('int64')
final_df['other_loans'] = final_df['other_loans'].astype('int64')

In [98]:
final_df.columns

Index(['uid', 'applied_contract_type', 'total_tl', 'complete_tl', 'active_tl',
       'amount_overdue_sum', 'loan_amount_avg', 'loan_amount_min',
       'loan_amount_max', 'num_times_delinquent',
       'months_since_recent_delinquency', 'automobile_loans', 'business_loans',
       'consumer_loans', 'mortgages_and_real_estate_loans', 'other_loans',
       'enquiry_count', 'max_enquiry_amount', 'time_since_last_enquiry',
       'business_loans_enquiries', 'consumer_loans_enquiries',
       'revolving_loans_enquiries',
       'mortgages_and_real_estate_loans_enquiries', 'other_loans_enquiries',
       'automobile_loans_enquiries'],
      dtype='object')

In [99]:
final_df.shape

(46127, 25)

In [96]:
def clubbing(col, threshold):
    if col>=threshold:
        return threshold
    else:
        return col
        
final_df['total_tl'] = final_df['total_tl'].apply(lambda x:clubbing(x,threshold=15))
final_df['complete_tl'] = final_df['total_tl'].apply(lambda x:clubbing(x,threshold=15))
final_df['active_tl'] = final_df['total_tl'].apply(lambda x:clubbing(x,threshold=5))

In [100]:
final_df.to_csv('../data/test/final_df.csv', index=False)